# 用 Tensorflow 实现 Multilayer Perceptron

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# parameters

learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# network parameters

n_hidden_1 = 256 # number of features at 1st layer
n_hidden_2 = 256 # number of features at 2nd layer
n_input = 784    # input data (img shape: 28 * 28)
n_classes = 10   # total classes (0~9 digits)

# graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [8]:
# create model

def multilayer_perceptron(x, weights, biases):
    layer1 = tf.add(tf.matmul(x, weights["h1"]), biases["b1"])
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.add(tf.matmul(layer1, weights["h2"]), biases["b2"])
    layer2 = tf.nn.relu(layer2)
    
    output_layer = tf.add(tf.matmul(layer2, weights["out"]), biases["out"])
    return output_layer

In [9]:
# store layers weight & bias

weights = {
    "h1": tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    "h2": tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    "out": tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    "b1": tf.Variable(tf.random_normal([n_hidden_1])),
    "b2": tf.Variable(tf.random_normal([n_hidden_2])),
    "out": tf.Variable(tf.random_normal([n_classes]))
}

In [11]:
# construct model

pred = multilayer_perceptron(x, weights, biases)

# Docs: https://www.tensorflow.org/api_docs/python/nn/classification#sparse_softmax_cross_entropy_with_logits
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
# Docs: https://www.tensorflow.org/api_docs/python/train/optimizers#AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize variables

init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
    sess.run(init)
    
    # training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            avg_cost += c / total_batch
        
        if epoch % display_step == 0:
            print("Epoch: {:04d} cost={:.9f}".format(epoch + 1, avg_cost))
    
    print("Optimization Finished")
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Docs: https://www.tensorflow.org/api_docs/python/array_ops/casting#cast
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy: {:.4f}".format(accuracy.eval({x: mnist.test.images, y: mnist.test.labels})))

Epoch: 0001 cost=149.835216827
Epoch: 0002 cost=39.800645334
Epoch: 0003 cost=24.847940483
Epoch: 0004 cost=17.078688671
Epoch: 0005 cost=12.129728342
Epoch: 0006 cost=8.944813778
Epoch: 0007 cost=6.656276842
Epoch: 0008 cost=5.005658170
Epoch: 0009 cost=3.592631458
Epoch: 0010 cost=2.675583695
Epoch: 0011 cost=2.155689024
Epoch: 0012 cost=1.616791521
Epoch: 0013 cost=1.193659379
Epoch: 0014 cost=1.008896908
Epoch: 0015 cost=0.782129565
Optimization Finished
Accuracy: 0.9478
